## Comparing Long Term Investment Strategies
### Karandeep Singh

# Investments are critical to long term financial security. Through the power of compound interest, investors can grow their wealth over time. However, there are many different investment strategies that can be used to grow wealth.

In this analysis, we will compare the benefits of investing in:
- Equities in the Largest Economy (US)
- Local Equities (Singapore)
- Emerging Market Equities (India)
- Safe Haven Assets (Gold)
- Low Risk Assets (US Treasury Bonds)
- Professional Management (Mutual Funds)

### The goal of this analysis is to determine which investment strategies are the most effective in growing wealth over time.

This analysis is split into 6 questions, over 3 parts.

**Part 1: Investment Vehicles**

1. What benefits does investing offer, compared to holding cash?
2. Which investment vehicles are the most effective in growing wealth over time?

**Part 2: Managing Risk**

3. Which investment vehicles are the most volatile?
4. Which investment vehicles offer the best risk-adjusted returns?

**Part 3: Understanding Stock Price Growth Drivers**

5. Does GDP growth translate to stock market growth?
6. What are the components of the NIFTY 50 Index?
7. Do bond yields affect stock market growth?

This analysis makes use of the following datasets:
- [SPDR S&P 500 ETF Trust (SPY) Historical Prices](https://finance.yahoo.com/quote/SPY/history?period1=1659308442&period2=1690844442&interval=1mo&filter=history&frequency=1mo&includeAdjustedClose=true)
- [FTSE Straits Times Index STI Historical Prices](https://www.wsj.com/market-data/quotes/index/SG/STI/historical-prices)
- [NIFTY Indices Dataset](https://www.kaggle.com/datasets/sudalairajkumar/nifty-indices-dataset)
- [NIFTY 50 (^NSEI)](https://finance.yahoo.com/quote/%5ENSEI/history)
- [NIFTY 50 Components List](https://www.nseindia.com/products-services/indices-nifty50-index)
- [Gold Spot Prices](https://www.gold.org/goldhub/data/gold-prices)
- [U.S. Department of the Treasury - Interest Rates Data CSV Archive](https://home.treasury.gov/interest-rates-data-csv-archive)
- [Fidelity Growth Company Fund Historical Quotes](https://www.marketwatch.com/investing/fund/fdgrx/download-data)
- [Singapore Department of Statistics - Consumer Price Index (CPI), 2019 As Base Year](https://tablebuilder.singstat.gov.sg/table/TS/M212881)
- [U.S. Bureau of Labor Statistics - Consumer Price Index for All Urban Consumers](https://www.bls.gov/cpi/tables/supplemental-files/)
- [Singapore Dollars to U.S. Dollar Spot Exchange Rate](https://fred.stlouisfed.org/series/DEXSIUS)
- [Indian Rupees to U.S. Dollar Spot Exchange Rate](https://fred.stlouisfed.org/series/EXINUS)
- [U.S. Gross Domestic Product (GDP)](https://fred.stlouisfed.org/series/GDP/)
- [India Gross Domestic Product (GDP)](https://fred.stlouisfed.org/series/MKTGDPINA646NWDB)

In [1]:
# Importing required libraries
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.ticker import FuncFormatter

### Loading and Preprocessing the Datasets

In [2]:
class ParsedDataFrames:
    spy_data = None
    btc_data = None
    sti_data = None
    nifty_data = None
    gold_data = None
    bond_data = None
    mf_data = None
    sg_cpi_data = None
    us_cpi_data = None
    gdp_data = None
    nifty_components = None

data = ParsedDataFrames()

The ***U.S. Gross Domestic Product*** dataset contains data about the United States' economic output. GDP is used to measure the size of the economy and is measured once per quarter. The GDP is represented in billions of dollars.

In [3]:
gdp_data = pd.read_csv('data/USA-GDP.csv', parse_dates=True)
gdp_data.columns = ['Date', 'USA']
gdp_data['Date'] = pd.to_datetime(gdp_data['Date'])
gdp_data.head()

,Date,USA
0,1947-01-01,243.164
1,1947-04-01,245.968
2,1947-07-01,249.585
3,1947-10-01,259.745
4,1948-01-01,265.742


The ***India Gross Domestic Product*** dataset contains data about India's economic output. The GDP is measured in USD and contains raw values. We need to divide the values to get the GDP in billions of USD for a fair comparison with the US GDP.

In [4]:
india_gdp_data = pd.read_csv('data/India-GDP.csv', parse_dates=True)
india_gdp_data.columns = ['Date', 'India']
india_gdp_data['India'] = india_gdp_data['India'] / 1000000000 # data is raw dollars, divide to get billions
india_gdp_data['Date'] = pd.to_datetime(india_gdp_data['Date'])

gdp_data = pd.merge(gdp_data, india_gdp_data, how="inner", on="Date")
data.gdp_data = gdp_data

data.gdp_data = data.gdp_data[data.gdp_data['Date'] >= '2000-01-01']
data.gdp_data = data.gdp_data[data.gdp_data['Date'] <= '2022-12-31']

data.gdp_data.tail()

,Date,USA,India
58,2018-01-01,20155.486,2702.929640
59,2019-01-01,21013.085,2835.606242
60,2020-01-01,21538.032,2671.595390
61,2021-01-01,22313.850,3150.306834
62,2022-01-01,24740.480,3385.089882


Later, we'll be using data in United States Dollars, Singapore Dollars and Indian Rupees.

The ***Singapore Dollars to U.S. Dollar Spot Exchange Rate*** and ***Indian Rupees to U.S. Dollar Spot Exchange Rate*** datasets allow us to compare this data correctly. They contain the exchange rates for USDSGD and USDINR respectively.

The USDINR dataset, however, does not contain values for every single day, and we thus need to forward fill them ourselves.

In [5]:
usd_sgd = pd.read_csv('data/DEXSIUS.csv')
usd_sgd.columns = ['Date', 'Close']

# Some values of Close are ".", so we need to convert them to NaN to be forward filled
usd_sgd['Close'] = usd_sgd['Close'].replace('.', np.nan)
usd_sgd['Close'] = usd_sgd['Close'].fillna(method='ffill') # Forward fill the NaN values
usd_sgd['Close'] = usd_sgd['Close'].astype(float) # Convert Close to float
usd_sgd['Date'] = pd.to_datetime(usd_sgd['Date']) # Convert Date to datetime

usd_sgd.tail()

,Date,Close
11101,2023-07-24,1.3300
11102,2023-07-25,1.3277
11103,2023-07-26,1.3266
11104,2023-07-27,1.3293
11105,2023-07-28,1.3305


In [6]:
usd_inr = pd.read_csv('data/EXINUS.csv', parse_dates=True)
usd_inr.columns = ['Date', 'Close']
usd_inr['Close'] = usd_inr['Close'].astype(float)

# Create a row for each day
usd_inr['Date'] = pd.to_datetime(usd_inr['Date'])
usd_inr = usd_inr.resample('D', on='Date').mean()
usd_inr = usd_inr.fillna(method='ffill')
usd_inr = usd_inr.reset_index()

usd_inr.tail()

,Date,Close
18409,2023-05-28,82.2977
18410,2023-05-29,82.2977
18411,2023-05-30,82.2977
18412,2023-05-31,82.2977
18413,2023-06-01,82.2252


The ***SPDR S&P 500 ETF Trust (SPY) Historical Prices*** dataset contains the historical prices of the SPY ETF. This is a fund that tracks the performance of the 500 largest companies in the USA. It is the most commonly used proxy for the entire US stock market.

We can discard all columns apart from the date and the adjusted close, which is the price of the ETF at the end of each trading day and adjusted for dividends and stock splits. A stock split is a process where a single share is split into multiple shares, which reduces the price of each share and can cause our data to be incorrect unless we adjust for it.

In [7]:
spy_data = pd.read_csv('data/SPY.csv')
spy_data = spy_data[['Date', 'Adj Close']]
spy_data.columns = ['Date', 'SPY']

# Date is formatted as yyyy-mm-dd, convert to datetime
spy_data['Date'] = pd.to_datetime(spy_data['Date'])

data.spy_data = spy_data
data.spy_data.head()

,Date,SPY
0,1993-01-29,25.029373
1,1993-02-01,25.207396
2,1993-02-02,25.260807
3,1993-02-03,25.527821
4,1993-02-04,25.634634


The ***FTSE Straits Times Index STI Historical Prices*** dataset contains the historical prices of the Straits Times Index (STI) from 2000 to 2019. Like SPY, STI is an index that can be used as a proxy for the Singapore stock market. It tracks the 30 largest companies listed on the Singapore Exchange (SGX).

STI has never had a stock split, so the price data does not need to be adjusted. However, we need to convert it to USD for a fair comparison with the other datasets.

In [8]:
sti_data = pd.read_csv('data/STI.csv')
sti_data = sti_data[['Date', ' Close']]
sti_data.columns = ['Date', 'STI']

# Date is formatted as mm/dd/yy, convert to datetime
sti_data['Date'] = pd.to_datetime(sti_data['Date'], format='%m/%d/%y')

# Convert to USD
sti_data['STI'] = sti_data['STI'] / usd_sgd['Close']

data.sti_data = sti_data
data.sti_data.head()

,Date,STI
0,2023-07-31,1612.030578
1,2023-07-28,1619.975973
2,2023-07-27,1604.528846
3,2023-07-26,1588.923077
4,2023-07-25,1578.746097


The ***NIFTY50*** is the flagship index on the National Stock Exchange of India Ltd. (NSE). It includes 50 of India's top companies and is a good proxy for the Indian stock market. We use the Indian stock market as our benchmark for emerging markets because of its size, liquidity, and ease of access for foreign investors.

While China's equity market is larger, foreign investors are not allowed to own Chinese stocks and instead invest in them through a special corporate structure called a variable interest entity (VIE). This is legally a grey area and investors have no ownership rights in the underlying companies. As a result, we do not consider China's equity market as a viable benchmark for emerging markets for this analysis, although some investors may choose to take a different view. For more on this topic, see [this article from CNN Business](https://edition.cnn.com/2021/08/08/investing/stocks-week-ahead/index.html).

This analysis uses two separate datasets for NIFTY50. This is because the first dataset only contains values from 2000-2020, while the second only contains values from 2007-2023. We need to add the rows from the second dataset to the first to create a dataset with values from 2000-2023.

The prices need to be converted to USD for a fair comparison with the other datasets.

In [9]:
nifty_data = pd.read_csv('data/NIFTY50.csv')
nifty_data = nifty_data[['Date', 'Close']]
nifty_data['Date'] = pd.to_datetime(nifty_data['Date'], format='%Y-%m-%d')
# Remove post-2021 data because it's not complete
nifty_data = nifty_data[nifty_data['Date'] < '2021-01-01']
data.nifty_data = nifty_data

nifty_data = pd.read_csv('data/^NSEI.csv')
# Remove all pre-2021 data
nifty_data = nifty_data[nifty_data['Date'] >= '2021-01-01']

# Format data to match NIFTY50.csv
nifty_data = nifty_data[['Date', 'Close']]
nifty_data['Date'] = pd.to_datetime(nifty_data['Date'], format='%Y-%m-%d')

# Combine datasets
nifty_data = pd.concat([data.nifty_data, nifty_data])
nifty_data = nifty_data.reset_index(drop=True)

nifty_data['Close'] = nifty_data['Close'] / usd_inr['Close']
nifty_data.columns = ['Date', 'NIFTY']

data.nifty_data = nifty_data
data.nifty_data.tail()

,Date,NIFTY
5863,2023-07-28,1301.709510
5864,2023-07-31,1308.848818
5865,2023-08-01,1307.507092
5866,2023-08-02,1293.791670
5867,2023-08-03,1284.190849


The ***NIFTY 50 Components List*** contains data about the individual companies that the NIFTY 50 index comprises.

In [10]:
nifty_components = pd.read_csv('data/ind_nifty50list.csv')

data.nifty_components = nifty_components
data.nifty_components.head()

,Company Name,Industry,Symbol,Series,ISIN Code
0,Adani Enterprises Ltd.,Metals & Mining,ADANIENT,EQ,INE423A01024
1,Adani Ports and Special Economic Zone Ltd.,Services,ADANIPORTS,EQ,INE742F01042
2,Apollo Hospitals Enterprise Ltd.,Healthcare,APOLLOHOSP,EQ,INE437A01024
3,Asian Paints Ltd.,Consumer Durables,ASIANPAINT,EQ,INE021A01026
4,Axis Bank Ltd.,Financial Services,AXISBANK,EQ,INE238A01034


The ***Gold Spot Prices*** dataset records gold prices (per ounce) from 1978 to 2023. Gold is often used as a hedge against inflation and is considered a safe haven asset due to its scarcity and its value in jewelry and electronics. This low risk profile makes gold a popular choice during times of economic uncertainty.

Unlike most of the other datasets, this is provided as a Microsoft Excel workbook with multiple sheets.

For maximum precision and granularity, we will use the "Daily" sheet, as this contains daily prices. The first 5 rows contain metadata, so we will skip them. The first 2 columns are empty and purely for formatting, so we will skip them as well. The remaining columns contain the Date, and the prices in various currencies. We will only use the USD column for a more consistent comparison with the other datasets.

In [11]:
gold_data = pd.read_excel('data/GoldSpotPrices.xlsx', sheet_name='Daily', skiprows=5, index_col=2)
gold_data = gold_data.drop(['Unnamed: 0', 'Unnamed: 1'], axis=1)
gold_data = gold_data.iloc[:, 0:1]
# The index is already in datetime format, but needs to be converted to a regular column
gold_data.reset_index(level=0, inplace=True)
gold_data.columns = ['Date', 'Gold']

data.gold_data = gold_data
data.gold_data.head()

The ***U.S. Department of the Treasury - Interest Rates Data CSV Archive*** contains data about US Treasury Bill interest rates. T-Bills are a financial instrument that the U.S Government uses to control the amount of money in circulation. Investors buy T-Bills from the government at a discount to the face value (the amount that the government will pay back when the T-Bill matures). The difference between the face value and the purchase price is the interest that the investor earns when the T-Bill matures.

T-Bills are considered risk-free investments because the US Government is extremely unlikely to default on its loans. They are sold at different lengths of time to maturity, from 4 to 52 weeks. Longer maturities generally pay higher interest rates, as investors demand more compensation for the risk of holding the investment for a longer period of time. We will hence consider only the 52 week T-Bill for our analysis as it is the longest maturity available.

However, there are two columns for 52 week bills - "52 WEEKS BANK DISCOUNT" and "52 WEEKS COUPON EQUIVALENT". T-Bills only pay out at the end of the investment, and this is what the former represents. The latter is the equivalent interest rate that the investor would earn if the T-Bill paid interest every 6 months. To avoid confusion, we will stick with the "52 WEEKS BANK DISCOUNT" column.

In [ ]:
bond_data = pd.read_csv('data/US-Treasury.csv')
bond_data['Date'] = pd.to_datetime(bond_data['Date'], format='%m/%d/%y')
bond_data = bond_data[['Date', '52 WEEKS BANK DISCOUNT']]

bond_data = bond_data.sort_values(by='Date')
bond_data = bond_data.reset_index(drop=True)
bond_data.dropna(inplace=True)

data.bond_data = bond_data
data.bond_data.head()

The ***Fidelity Growth Company Fund Historical Quotes*** dataset contains the historical stock prices of the Fidelity Growth Company Fund from 2000 to 2022. The Fidelity Growth Company Fund is a professionally managed mutual fund that invests in companies that are expected to grow faster than the rest of the market. Investors invest in mutual funds hoping to earn higher returns with the help of professional fund managers.

The dataset is split into 22 CSV files, one for each year.

In [ ]:
mf_files = []
for i in range (0, 23):
    # Files are named for the last 2 digits of the year, make sure to add 0 to single digit years
    mf_files.append(pd.read_csv('data/Fidelity/' + str(i).zfill(2) + '.csv'))
mf_data = pd.concat(mf_files, ignore_index=True)
mf_data = mf_data[['Date', 'Close']]
mf_data['Date'] = pd.to_datetime(mf_data['Date'], format='%m/%d/%Y')
mf_data.columns = ['Date', 'MF']

data.mf_data = mf_data
data.mf_data.head()

The ***Singapore Department of Statistics - Consumer Price Index (CPI), 2019 As Base Year*** dataset contains the monthly consumer price index (CPI) from 1961 to 2023. CPI is a measure of inflation, and is calculated by measuring the change in price of a fixed basket of goods and services, including food, power, and rent.

If an investor holds cash without earning interest and the CPI is rising, the investor's purchasing power is decreasing, which can be considered as a loss of money.

The first 10 rows of the dataset contain metadata and will be skipped. This dataset is arranged such that the dates are in columns and the CPI values are in rows. For consistency, the dataset will be transposed so that the dates are in rows and the CPI values are in columns. We will also consider only the "All Items" category, which is the overall CPI.

CPI data is structured as relative values against a base of 100, which is the CPI value in the base year. The base value for this dataset was in 2021, but to make calculations easier later, we will convert the base year to December 2022.

In [ ]:
sg_cpi_data = pd.read_csv('data/SG-CPI.csv', skiprows=10)
sg_cpi_data = sg_cpi_data.T # Transpose
sg_cpi_data = sg_cpi_data.iloc[:, :1] # Keep only the first column (All Items)
sg_cpi_data.columns = sg_cpi_data.iloc[0] # Set column names to the first row
sg_cpi_data = sg_cpi_data.iloc[1:, :] # Remove the first row
sg_cpi_data.index = sg_cpi_data.index.str.strip()
sg_cpi_data.index = pd.to_datetime(sg_cpi_data.index, format='%Y %b')
sg_cpi_data.reset_index(inplace=True)
sg_cpi_data.columns = ['Date', 'CPI']
sg_cpi_data['CPI'] = sg_cpi_data['CPI'].astype(float)

# Remove post-2022 data and reindex
sg_cpi_data = sg_cpi_data.loc[sg_cpi_data['Date'] <= '2022-12-01']
sg_cpi_data.reset_index(drop=True, inplace=True)

# Rebase so that the base of 100 is December 2022
december_2022_cpi = sg_cpi_data.loc[sg_cpi_data['Date'] == '2022-12-01', 'CPI'].values[0]
sg_cpi_data['CPI'] = (sg_cpi_data['CPI'] / december_2022_cpi) * 100

data.sg_cpi_data = sg_cpi_data
data.sg_cpi_data.head()

The ***U.S. Bureau of Labor Statistics - Consumer Price Index for All Urban Consumers*** dataset contains U.S. CPI data from 1913 to 2023. Since many of our datasets are in USD, it is important to also track how the USD is changing in value over time.

This dataset is provided as a Microsoft Excel workbook, where the first 11 rows are metadata and will be skipped. The last two columns contain the overall CPI for the first and second half of each year, and will also be dropped.

The base value for this dataset was in the 1980s, but to make calculations easier later, we will convert the base year to December 2022.

In [ ]:
us_cpi_data = pd.read_excel('data/USA-CPI.xlsx', skiprows=11, usecols=range(13))
us_cpi_data = us_cpi_data.melt(id_vars=['Year'], var_name='Month', value_name='CPI')
us_cpi_data['Date'] = us_cpi_data['Year'].astype(str) + '-' + us_cpi_data['Month']
us_cpi_data['Date'] = pd.to_datetime(us_cpi_data['Date'], format='%Y-%b')
us_cpi_data = us_cpi_data.drop(columns=['Year', 'Month'])
us_cpi_data.columns = ['CPI', 'Date']
us_cpi_data['CPI'] = us_cpi_data['CPI'].astype(float)

# Remove post-2022 data
us_cpi_data = us_cpi_data[us_cpi_data['Date'] <= '2022-12-01']
# Sort by date
us_cpi_data = us_cpi_data.sort_values(by='Date')
# Reindex
us_cpi_data = us_cpi_data.reset_index(drop=True)

# Rebase so that the base of 100 is December 2022
december_2022_cpi = us_cpi_data[us_cpi_data['Date'] == '2022-12-01']['CPI'].values[0]
us_cpi_data['CPI'] = us_cpi_data['CPI'] / december_2022_cpi * 100
us_cpi_data.tail()

data.us_cpi_data = us_cpi_data

Finally, we can merge the dataframes together.

In [ ]:
# merge data.spy_data, data.sti_data, and data.nifty_data on date

asset_data = pd.merge(data.spy_data, data.sti_data, on='Date', how='inner')
asset_data = pd.merge(asset_data, data.nifty_data, on='Date', how='inner')
asset_data = pd.merge(asset_data, data.gold_data, on='Date', how='inner')
asset_data = pd.merge(asset_data, data.mf_data, on='Date', how='inner')
asset_data.head()

### 1. What benefits does investing offer, compared to holding cash?

Before analysing different investments, we need to understand why we should invest in the first place. In this section, we will analyse the issue that plagues all economic participants: inflation.

In [ ]:
# save 1000 USD in each currency every month, account for inflation and reduce previous values
currencies = [[], []] # SGD, USD
sgd_values = []
raw_values = []

for year in range(2000, 2023):
    for month in range(1, 13):
        rate = usd_sgd.loc[(usd_sgd['Date'].dt.year == year) & (usd_sgd['Date'].dt.month == month)]['Close'].values[0] # get USD/SGD rate for month
        converted_sgd = 1000 * rate # get val of 1000 USD in SGD
        sgd_value_with_inflation = converted_sgd * ((data.sg_cpi_data.loc[(data.sg_cpi_data['Date'].dt.year == year) & (data.sg_cpi_data['Date'].dt.month == month)]['CPI'].values[0])/100)
        if len(currencies[0]) == 0:
            raw_values.append(1000)
            sgd_values.append(sgd_value_with_inflation)
            currencies[1].append(1000 * ((data.us_cpi_data.loc[(data.us_cpi_data['Date'].dt.year == year) & (data.us_cpi_data['Date'].dt.month == month)]['CPI'].values[0])/100))
        else:
            raw_values.append(1000 + raw_values[-1])
            sgd_values.append((sgd_value_with_inflation) + sgd_values[-1])
            currencies[1].append((1000 * ((data.us_cpi_data.loc[(data.us_cpi_data['Date'].dt.year == year) & (data.us_cpi_data['Date'].dt.month == month)]['CPI'].values[0])/100)) + currencies[1][-1])
        currencies[0].append(sgd_values[-1] / rate) # convert SGD to USD

dates = pd.date_range(start='2000-01-01', end='2022-12-31', freq='M') # every month from 2000-2022

sns.set_style('darkgrid')
sns.lineplot(x=dates, y=currencies[0], label='SGD')
sns.lineplot(x=dates, y=currencies[1], label='USD')
sns.lineplot(x=dates, y=raw_values, label='Contributions (without inflation)')

plt.ylim(0, 300000)
plt.xlim(pd.Timestamp('2000-01-01'), pd.Timestamp('2022-12-31'))
plt.xticks(rotation=25)
plt.title('Growth of $1000 Monthly Investment (2000-2022)')
plt.gca().yaxis.set_major_formatter(FuncFormatter(lambda x, _: '${:,.0f}'.format(x/1000) + 'k')) # Format y-axis ticks in 100k increments
plt.show()

It is important to have healthy savings in case of emergencies, and also to prepare for retirement. If a person puts $1000 into a savings account that pays 0% interest rate, they might make the mistake of assuming that they're building a strong retirement fund, but this is not the case.

From 2000-2022, there were 276 months. Therefore, it would be reasonable to expect that the saver would end with 276,000 USD. However, the saver ended with 207k USD. This is due to inflation eroding their buying power. The Consumer Price Index is a measure of inflation, and if we look at the data, we can see that the CPI has been increasing over the years. This means that the saver's money is worth less and less over time. The original $100 that they started with is now worth less than $700.

This issue is not exclusive to the U.S Dollar. The Singapore Dollar has also been depreciating against inflation. Despite investing 276k USD worth of SGD, the investor ended with just 237k USD. The reason for the amount being larger than the USD is that the Singapore Dollar has been appreciating against the USD since 2000, offsetting some of the inflation. However, it should be noted that the SGD still failed to keep up with inflation.

The solution to this problem is to invest in assets that appreciate faster than inflation. In the coming sections, we analyse different assets and their performance against inflation.

# 2. Which investment vehicles are the most effective in growing wealth over time?



In [ ]:
# Assume the investor started investing in 2000, and contributed 1000 USD every month.

total_shares = [[], [], [], [], []]
total_value = [[], [], [], [], []]

assets = ['SPY', 'STI', 'NIFTY', 'Gold', 'MF']

for i, asset in enumerate(assets):
    for year in range(2000, 2023):
        for month in range(1, 13):
            # Get row from asset_data where year and month matches the Date column
            row = asset_data.loc[(asset_data['Date'].dt.year == year) & (asset_data['Date'].dt.month == month)]
            # Calculate number of shares purchased with $1000
            if len(total_shares[i]) == 0:
                num_shares = 1000 / row[asset].values[0]
            else:
                num_shares = total_shares[i][-1] + (1000 / row[asset].values[0])
            total_shares[i].append(num_shares)
            # Calculate total value of shares
            total_value[i].append(num_shares * row[asset].values[0])

palette = sns.color_palette("bright", 8)
sns.set_style('darkgrid')

sns.lineplot(x=dates, y=total_value[0], label='S&P 500')
sns.lineplot(x=dates, y=total_value[1], label='Straits Times Index')
sns.lineplot(x=dates, y=total_value[2], label='NIFTY 50')
sns.lineplot(x=dates, y=total_value[3], label='Gold')
sns.lineplot(x=dates, y=total_value[4], label='Fidelity Growth Company Fund')
sns.lineplot(x=dates, y=currencies[0], label='SGD')
sns.lineplot(x=dates, y=currencies[1], label='USD')
sns.lineplot(x=dates, y=raw_values, label='Contributions (without inflation)')

plt.ylim(0, 1400000)
plt.xlim(pd.Timestamp('2000-01-01'), pd.Timestamp('2022-12-31'))
plt.xticks(rotation=25)
plt.title('Growth of $1000 Monthly Investment (2000-2022)')
plt.gca().yaxis.set_major_formatter(FuncFormatter(lambda x, _: '${:,.0f}'.format(x/1000) + 'k')) # Format y-axis ticks in 100k increments
plt.show()

# bar chart of total value
sns.barplot(x=['SPY', 'STI', 'NIFTY', 'Gold', 'MF', 'SGD', 'USD'], y=[total_value[0][-1], total_value[1][-1], total_value[2][-1], total_value[3][-1], total_value[4][-1], currencies[0][-1], currencies[1][-1]], palette=palette)
plt.gca().yaxis.set_major_formatter(FuncFormatter(lambda x, _: '${:,.0f}'.format(x/1000) + 'k')) # Format y-axis ticks in 100k increments

for index, value in enumerate([total_value[0][-1], total_value[1][-1], total_value[2][-1], total_value[3][-1], total_value[4][-1]]):
    plt.text(index, value, '${:,.0f}'.format(value/1000) + 'k', ha='center', va='bottom')
for index, value in enumerate([currencies[0][-1], currencies[1][-1]]):
    plt.text(index+5, value, '${:,.0f}'.format(value/1000) + 'k', ha='center', va='bottom')
plt.title('Final Value after 22 Years of Investment')
plt.show()

From the results, we can see that the Straits Times Index consistently underperformed all other non-cash assets in the portfolio, but it did beat inflation. This should be expected, as The Straits Times Index consists of mainly banks and real estate companies. These companies are not expected to perform well in a low interest rate environment, like the one from 2010 to 2022.

For better results, the investor could have invested in Gold, which benefitted from uncertainty in the market in 2008-2012 (owing to the Global Financial Crisis) and 2020-2022 (owing to the COVID-19 pandemic). The investor would have considerably more money if they had invested in Gold instead of the Straits Times Index, at 706,000 USD.

Our selected Mutual Fund did even better, ending at 864,000 USD. This can be attributed to the fact that the Fidelity Growth Company Fund invests in companies that are expected to grow at a faster rate than the market. This is a good strategy to use in a low interest rate environment, as investors are more willing to take on risk in order to get higher returns.

However, despite being professionally managed, the Mutual Fund still failed to beat the S&P 500, which ended at 960,000 USD. The difference is even larger if we account for the management fee that Mutual Funds tend to charge. This shows that even professionals are unable to consistently beat the market.

Emerging Markets were the best performing asset in the portfolio, ending at 973,000 USD. This can be attributed to the rapid economic advancements in India. India was also not hit as hard by the 2008 Global Financial Crisis, as it was not as integrated into the global economy as it is today. This allowed India to continue growing, while the U.S was in a recession. Similarly, the U.S entered a smaller recession in late 2021 due to high inflation, but India was not affected as much.

**We can thus conclude that investing is a valuable and important tool to grow your wealth, and any intelligent investment strategy should outperform inflation. ETFs are the most effective way to grow your wealth, particularly the NIFTY50 ETF.**

However, some investments are more volatile than others. Some investors may choose to settle for lower returns in exchange for lower volatility and better peace of mind. In the next section, we will look at the volatility of each asset class in the portfolio.

# 3. Which investment vehicles are the most volatile?

In [ ]:
# Calculate the daily returns for each asset
returns = asset_data.drop('Date', axis=1).pct_change()

# Calculate the standard deviation (volatility) for each asset
volatility = returns.std()

# Set the style for the plots using seaborn
sns.set(style="whitegrid")

# Create box plots for each asset on the same frame
plt.figure(figsize=(10, 6))
sns.boxplot(data=returns, orient="h")
plt.title('Volatility of Different Assets')
plt.xlabel('Returns')
plt.tight_layout()

plt.show()

From the box plot above, we can see that the outliers in the NIFTY50 index were the most extreme and the interquartile range was quite large. This is because it contains only 50 companies, which means that the price movements of each company have a greater influence on the overall index, compared to the S&P500.

Gold has the smallest interquartile range, which means that the price movements of gold are more stable than the other two assets. This is because gold is a safe haven asset.

The Fidelity Growth Company Fund has the largest interquartile range, indicating greater volatility. This is because it invests in growth stocks, which tend to be smaller companies with less trading volume, which means it is easier to move the price significantly.

**To conclude, the mutual fund was the most volatile and gold was the least volatile. Investors need to decide whether they are willing to accept lower returns in exchange for lower volatility. To help make this decision, we will now investigate risk-adjusted returns.**

# 4. Which investment vehicles offer the best risk-adjusted returns?

In [ ]:
total_returns = asset_data.drop('Date', axis=1).pct_change()
trading_days = len(asset_data['Date']) / 22 # Total trading days / 22 years
annualized_return = total_returns.mean() * trading_days # Annualized return = daily mean * trading days
################################################
# Calculate the risk-free rate             #####
################################################
bond_data = data.bond_data.copy()

# Resample the data to 52-week intervals
# (we're looking at 52 week T-Bills)
bond_data.set_index('Date', inplace=True)
bond_data_compounded = (1 + bond_data / 100).resample('52W').prod().fillna(1)

# Total compounded interest 
total_compounded_interest = bond_data_compounded.cumprod().iloc[-1] - 1
num_years = (bond_data.index[-1] - bond_data.index[0]).days / 365

# Average yearly return
average_yearly_return = (total_compounded_interest + 1) ** (1 / num_years) - 1
average_yearly_return = (average_yearly_return / 100)[0]
print("Average yearly return:", average_yearly_return)
################################################

risk_free_rate = average_yearly_return

volatility = total_returns.std() # Std dev of daily returns
sharpe_ratio = (annualized_return - risk_free_rate) / volatility

sns.set(style="darkgrid")
plt.figure(figsize=(4, 4))
sns.scatterplot(x=volatility, y=sharpe_ratio, hue=sharpe_ratio.index)
plt.title('Scatter Plot of Sharpe Ratio vs. Volatility')
plt.xlabel('Volatility (Lower is Better)')
plt.ylabel('Sharpe Ratio (Higher is Better)')
plt.legend()
plt.tight_layout()

plt.show()

To determine the risk-adjusted return of our investments, we calculate their Sharpe ratio. The Sharpe ratio divides a portfolio's excess returns (the returns above the risk-free rate, which we approximate with 52-week Treasury bills) by the portfolio's volatility (quantified by the standard deviation of returns). We can then visualise the results on a scatter plot, with the x-axis representing the volatility and the y-axis representing the excess returns.

A higher Sharpe ratio indicates a better risk-adjusted return, while 0 or lower indicates a negative risk-adjusted return, and the investor would have been better off investing in 52-week Treasury bills instead.

From the plot above, we can observe that Gold has the highest Sharpe ratio and lowest volatility, making it the best investment choice for risk-averse investors. The S&P 500 has a relatively high Sharpe ratio and low volatility, making it the best investment choice for the average investor. The NIFTY50 has a higher Sharpe ratio than the S&P 500, but also a much higher volatility, making it the best investment choice for risk-seeking investors. This could be why the NIFTY50 has the highest returns. The STI has a negative Sharpe ratio, making it the worst investment choice.

**In conclusion, the best risk-adjusted investment vehicles are Gold and the S&P 500. For investors seeking only to maximise returns, the NIFTY50 is the best choice. After considering the relatively small difference in total returns between SPY and NIFTY50 (as calculated in the previous section), we can conclude that the best investment choices are Gold or SPY, depending on the investor's risk tolerance.**

In the next part, we will try to understand what drives the returns of these investment vehicles.

# 5. Does GDP growth translate to stock market growth?

In [ ]:
merged_data = asset_data[['SPY', 'NIFTY']]
merged_data = pd.merge(data.gdp_data, merged_data, how='inner', left_index=True, right_index=True)

fig, axes = plt.subplots(1, 2, figsize=(15, 6))

sns.scatterplot(ax=axes[0], x='USA', y='SPY', data=merged_data)
axes[0].set_title('Correlation between USA GDP and SPY Stock Prices')

sns.scatterplot(ax=axes[1], x='India', y='NIFTY', data=merged_data)
axes[1].set_title('Correlation between India GDP and NIFTY Stock Prices')   
axes[0].annotate('Correlation: {:.2f}'.format(merged_data['USA'].corr(merged_data['SPY'])), xy=(0.1, 0.9), xycoords='axes fraction')
axes[1].annotate('Correlation: {:.2f}'.format(merged_data['India'].corr(merged_data['NIFTY'])), xy=(0.1, 0.9), xycoords='axes fraction')

plt.tight_layout() # Subplot spacing
plt.show()

In the scatter plot of GDP against stock market data above, we can observe correlation between GDP and the S&P 500 of 0.73. This is a strong correlation, and indicates that GDP may be a good predictor of stock prices too.

Interestingly, the correlation between the GDP of India and the NIFTY50 is -0.73. This means that the Indian stock market is negatively correlated with the GDP of India, which is a result that goes against intuition. Despite verifying the data manually, the results are the same. [Others have also observed this phenomenon](https://marcellus.in/blogs/nifty-indian-gdp-a-complete-breakdown-in-relationship/), and have attributed it to the NIFTY50 no longer being a good indicator of the Indian economy.

Despite its lack of correlation with the Indian economy, the NIFTY50 remains an important index to consider when investing in India due to its high popularity and liquidity.

**Nonetheless, we must conclude that GDP is an important metric to consider, but not always a good predictor of stock prices.** 

As our aim is now to understand what drives stock prices, we will now consider the impact of bond yields.

# 6. What are the components of the NIFTY 50 Index?

In [ ]:
industries = data.nifty_components['Industry'].unique()

industry_count = data.nifty_components['Industry'].value_counts()

plt.figure(figsize=(12,10))
plt.pie(industry_count, autopct='%1.1f%%', startangle=90)
plt.axis('equal')
plt.legend(labels=industries)
plt.title('Distribution of Industries in NIFTY 50', fontsize=18, fontweight='bold', loc='right')
plt.show()

The pie chart shows that the largest sectors of the NIFTY 50 are Metals & Mining, Services, and Healthcare. Information Technology is only 2% of the NIFTY 50, but is much more prevalent in the Indian economy. The IT sector also tends to grow much faster than other sectors, particularly in the past 12 years of low interest rates.

**This explains why the returns of the NIFTY 50 do not correlate with India's GDP - the NIFTY 50 is not representative of the Indian economy.**

# Final Conclusion

To summarise the most important findings of this project, we can say that:
- Investing your money is very important to avoid losing buying power over time.
- Assets like gold are less risky than stocks, but they also have a lower return.
- Gold is the best low-risk asset to invest in, while the S&P 500 is the best moderate-risk investment.
- GDP is not necessarily a good indicator of the stock market performance.
- The NIFTY 50 index is not a good indicator of the Indian economy, but outperforms all the other investments in this analysis.

### Important Considerations
- In our analysis, we assumed that the investor paid $0 in fees and commissions, and that they had the ability to purchase fractional shares. However, zero commission trading and fractional shares are both relatively new developments.
- We did not consider dividends in our analysis. Dividends are a way for companies to return profits to shareholders and can sometimes be significant, but they are not considered in this analysis as they can also bring additonal tax implications, which are outside the scope of this analysis.
- Past performance is not indicative of future results. While we have made every effort to understand the historical performance of the stocks in our analysis, it is not guaranteed that they will perform in the same way in the future, so we are at best operating on educated assumptions. Ideally, we would update our analysis with new data on a regular basis to ensure that our assumptions are still valid.